In [ ]:
# code sketch:
# 1. small subset of songs, get lyrics / info from spotify / genius api
#     turn lyrics coupled w metadata into vector using BERT or something
#.    store in Pinecone? FAISS
# 2. user input > gpt 4 / langchain to get semantic vector rep. of that
#.     compare using cosine sim., closest 10 get suggested


In [ ]:
# SPOTIFY / GENIUS END:
#.   (first, parse user input, find vector vs. "energy" and other things that could narrow the search)
#    pick 100 random songs that work > get their metadata and send to genius > extract lyrics, save as text files
#          if no lyrics, reject

In [ ]:
# LANGCHAIN:
#   turn the lyrics into vectors > store in pinecone
#   user input from gpt into langchain > comapre to vector store
#   Find top 10 songs in song names > send back to spotify to pull up those songs as playable

In [2]:
# import all necessary files
from setup import *
from spotify_handler import *

In [ ]:
trackList = [] #empty tracklist array we're gonna fill
songNumber = 10000 # number of songs we're gonna sample from randomly
pbar = tqdm(total = songNumber) #progress bar init
query_offset = 600

trackList = build_track_list(songNumber, query_offset)

while len(trackList) <= songNumber: 
    ranNum = random.randint(1,600) # make sure the offset is a random one so there's never repeats, and they aren't all super pop. new.
    query = get_random_search() #use the function to get a random search on a letter, ie. e% so any song that starts with that letter
    track = sp.search(query, limit=1, offset = ranNum, type='track') #query spotify
    trackURI = track['tracks']['items'][0]['uri']
    trackURI = trackURI.replace('spotify:album:', '')
    trackName = track['tracks']['items'][0]['name']
    trackArtist = track['tracks']['items'][0]['artists'][0]['name']
    audio_features = sp.audio_features(trackURI)
    instrumentalness = audio_features[0]['instrumentalness'] #does this have lyrics?
    
    if instrumentalness <= 0.5: #if no lyrics, reject and get a new one
        sleep(0.1)
        pbar.update(1)
        array = [trackName, trackArtist]
        trackList.append(array)

        track = None


In [ ]:
# GENIUS PART , now we take the song lists and download the lyrics for each one into separate text files for parsing with LangCHAIN:
# going to use lyricsgenius which is a python wrapper for GENIUS API
genius = Genius(token)


genius = lyricsgenius.Genius("my_token")  # replace with your token
genius.verbose = False

#custom get song lyrics function, with timeout behavior
def get_song_lyrics(title, artist, max_retries=5, delay_between_retries=10):
    for attempt in range(max_retries):
        try:
            song = genius.search_song(title=title, artist=artist)
            if song is not None:
                return song
            else:
                print(f"No lyrics found for {title} by {artist}.")
                return None
        except Timeout:
            if attempt < max_retries - 1:  # it's not the last attempt
                print(f"Timeout occurred when getting lyrics for {title} by {artist}. "
                      f"Waiting {delay_between_retries} seconds before retrying...")
                time.sleep(delay_between_retries)
                continue
            else:  # it's the last attempt
                print(f"Still experiencing timeouts after {max_retries} attempts. "
                      f"Giving up on getting lyrics for {title} by {artist}.")
                return None
        except Exception as e:
            print(f"An error occurred: {e}")
            return None

def sanitize_filename(name):
    for char in ['/', '\\', ':', '*', '?', '"', '<', '>', '|']:
        name = name.replace(char, '_')
    return name

for i in tqdm(trackList):
    title = sanitize_filename(i[0])
    artist = sanitize_filename(i[1])
    song = get_song_lyrics(title, artist,5,10)
    if song != None and "[Verse" in song.lyrics:
        lyrics = song.lyrics
        with open('lyrics/%s_%s.txt'%(title,artist), 'w') as f:
            f.write(lyrics)
            

In [ ]:
#LANGCHAIN TIME
# here gonna take each lyric file and summarize it / vectorize it and store in pinecone
# initialize langchain / openAI

embeddings = OpenAIEmbeddings(openai_api_key = openai_api_key)
llm = OpenAI(temperature=0)
chain = load_summarize_chain(llm, chain_type="stuff")

docs=[]
summary = []

for filename in tqdm(os.listdir(directory)):
    if filename.endswith(".txt"):  # check if the file is a .txt file
        file_path = os.path.join(directory, filename)
        # now you can do something with the file
        loader = TextLoader(file_path)
        doc = loader.load()
        docs.append(doc)
for i in tqdm(docs):
    summary.append(chain.run(i))

        

In [ ]:
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment='asia-northeast1-gcp'
)

index_name="spotifysemantic"

for i in tqdm(docs):
    docsearch=Pinecone.from_documents(i, embeddings, index_name=index_name)





In [ ]:
docsearch = Pinecone.from_existing_index(index_name, embeddings)
query = "songs for the best day of my life"
ans = docsearch.similarity_search(query)

for i in ans:
    print(i.metadata)